In [33]:
import pickle
import networkx as nx
import numpy as np

In [25]:
with open('GOT-characters-raw.pickle','rb') as fp:
    characters = pickle.load(fp)
len(characters.keys())

340

In [26]:
# set() : Créer des éléments unique
entities = set()
for chapter in characters.values():
    for entity in chapter:
        name = entity[0]
        entities.add(name)
print(len(entities))

5522


In [27]:
GOT = nx.Graph()
GOT.add_nodes_from(list(entities))
print(nx.info(GOT))

Name: 
Type: Graph
Number of nodes: 5522
Number of edges: 0
Average degree:   0.0000


In [28]:
chapter = characters['1–1-Bran']
# sort: tri en ordre croissant
# pour chaque key, prendre l'élément 1 de la clé puis trier les ensembles avec sort
chapter.sort(key=lambda a:a[1],reverse=False)

#print(chapter)
potential_edges = []
for i in range(0,len(chapter)): # prend le 1er de la liste
    name1,start1,end1 = chapter[i]
    for j in range(i+1,len(chapter)):
        name2,start2,end2 = chapter[j]
        if name2 != name1:
            potential_edges.append((name1,name2,{'dist':start2-end1}))
#print(len(potential_edges))
#print(potential_edges)

In [36]:
edges = {}
for name1,name2,value in potential_edges:
    if (name1,name2) not in edges.keys():
        if (name2,name1) not in edges.keys():
            edges[(name1,name2)] = {'values':[value['dist']]}
        else:
            edges[(name2,name1)]['values'].append(value['dist'])
    else:
        edges[(name1,name2)]['values'].append(value['dist'])

for k,v in edges.items():
    # 25: 25%
    # percentile: écarts types
    #edges[k]['weight']=np.percentile(v['values'],25)
    edges[k]['weight']=len([a for a in v['values'] if a < 1000])
    print(k,edges[k]['weight'])
    
#edges

('Bran', 'Robb') 36
('Bran', 'Mance Rayder') 1
('Bran', 'Old Nan') 3
('Bran', 'Jon') 23
('Bran', 'Stark') 6
('Bran', 'Eddard Stark') 2
('Bran', 'Theon Greyjoy') 6
('Bran', 'Jory Cassel') 3
('Bran', 'Robert') 4
('Bran', 'Eddard') 0
('Bran', 'North') 0
('Bran', 'Jon Snow') 0
('Bran', 'Greyjoy') 4
('Bran', 'Theon') 0
('Bran', 'Ass') 0
('Bran', 'Targaryen') 2
('Bran', 'Starks') 1
('Bran', 'Jory') 8
('Bran', 'Ice') 1
('Bran', 'Direwolves') 0
('Bran', 'Hullen') 0
('Bran', 'Harwin') 0
('Bran', "Ser Rodrik's") 0
('Bran', 'Rickon') 0
('Bran', 'Snow') 0
('Bran', 'Desmond') 1
('Robb', 'Mance Rayder') 3
('Robb', 'Old Nan') 7
('Robb', 'Jon') 113
('Robb', 'Stark') 15
('Robb', 'Eddard Stark') 1
('Robb', 'Theon Greyjoy') 19
('Robb', 'Jory Cassel') 7
('Robb', 'Robert') 4
('Robb', 'Eddard') 1
('Robb', 'North') 1
('Robb', 'Jon Snow') 9
('Robb', 'Greyjoy') 23
('Robb', 'Theon') 1
('Robb', 'Ass') 4
('Robb', 'Targaryen') 3
('Robb', 'Starks') 4
('Robb', 'Jory') 41
('Robb', 'Ice') 6
('Robb', 'Direwolves') 4
('